In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
from datasets import load_dataset
#load datasetnya dari huggingface
ds = load_dataset("Jing1113/interview_dialog_single_turn")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1197 [00:00<?, ? examples/s]

In [ ]:
import pandas as pd
#membaca dataset ke pandas dataframe
df = pd.DataFrame(ds["train"])

In [ ]:
df

,text
0,<s><INST> Hello. </INST> Hello! Thank you for ...
1,<s><INST> Hello! I have a Bachelor's degree in...
2,"<s><INST> Certainly! In a previous role, I was..."
3,<s><INST> When identifying KPIs for a business...
4,<s><INST> I tailor my communication style to t...
...,...
1192,<s><INST> During a marketing campaign for a pr...
1193,<s><INST> I stay motivated and inspired in my ...
1194,"<s><INST> In a previous role, I encountered a ..."
1195,<s><INST> To keep up with evolving technologie...


In [ ]:
#ubah menjadi 2 kolom untuk pertanyaan dan juga untuk jawaban
def extract_instruction_answer(text):
    instruction = text.split("<INST>")[1].split("</INST>")[0].strip()
    answer = text.split("</INST>")[1].split("</s>")[0].strip()
    return pd.Series([instruction, answer])

df[['Instruction', 'Answer']] = df['text'].apply(extract_instruction_answer)

df = df.drop(columns=['text'])


In [ ]:
df

,Instruction,Answer
0,Hello.,Hello! Thank you for joining me today. Let's s...
1,Hello! I have a Bachelor's degree in Business ...,That's great to hear! Can you tell me about a ...
2,"Certainly! In a previous role, I was tasked wi...",That sounds like a significant achievement! Ca...
3,"When identifying KPIs for a business, I start ...",That's a thorough approach to using KPIs effec...
4,I tailor my communication style to the audienc...,That's a valuable skill to have as a business ...
...,...,...
1192,During a marketing campaign for a product laun...,That's a great example of handling unexpected ...
1193,I stay motivated and inspired in my marketing ...,That's a wonderful source of motivation and pa...
1194,"In a previous role, I encountered a situation ...",That's a great example of turning a challengin...
1195,To keep up with evolving technologies in the d...,That's a great utilization of marketing automa...


In [ ]:
#ubah ke format finetune untuk gpt
import json


formatted_conversations = [
    {
        "messages": [
            {"role": "system", "content": "You are an interviewer for a company, asking questions and providing feedback."},
            {"role": "user", "content": row["Instruction"]},
            {"role": "assistant", "content": row["Answer"]}
        ]
    }
    for _, row in df.iterrows()
]

output_conversation_file_path = 'interview_conversations_finetune_dataset.jsonl'
with open(output_conversation_file_path, 'w') as f:
    for conversation in formatted_conversations:
        json.dump(conversation, f)
        f.write('\n')
